In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [26]:
data = pd.read_csv("/Users/jmadu1/Documents/healthcare-dataset-stroke-data.csv")
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [27]:
#declare X and y
X = data.drop(columns='stroke')
y = data['stroke']

In [28]:
print(f'The sum of all duplicated data is {data.duplicated().sum()}. \n')

print(f'Total null values per feature: \n {data.isnull().sum().sort_values(ascending=False)}')


The sum of all duplicated data is 0. 

Total null values per feature: 
 bmi                  201
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
smoking_status         0
stroke                 0
dtype: int64


In [29]:
print("Percentage of null values per feature: \n")
data.isnull().sum().sort_values(ascending=False)/len(data)


Percentage of null values per feature: 



bmi                  0.039335
id                   0.000000
gender               0.000000
age                  0.000000
hypertension         0.000000
heart_disease        0.000000
ever_married         0.000000
work_type            0.000000
Residence_type       0.000000
avg_glucose_level    0.000000
smoking_status       0.000000
stroke               0.000000
dtype: float64

NaNs are only 3% of data, this is fine. bmi being NaN means data was not retrieved for random patient listed.


In [30]:
data.rename(columns={"hypertension": "ATP2B1"})

,id,gender,age,ATP2B1,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [32]:
from sklearn.dummy import DummyRegressor #Regressor that makes predictions using simple rules.

baseline_model = DummyRegressor(strategy="mean") # Baseline
baseline_model.fit(X_train, y_train) # Calculate value for stratgy
baseline_model.score(X_test, y_test) # Score model based on consistently predicting the strategy

-0.0004402073298805931

In [33]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(baseline_model, X, y, cv=5)

In [34]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_score
0,0.002067,0.001021,-0.322122
1,0.001651,0.000562,0.000000
2,0.001176,0.000542,0.000000
3,0.001380,0.000734,0.000000
4,0.001375,0.000573,0.000000


In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

num_transformer = make_pipeline(SimpleImputer(), RobustScaler())
cat_transformer_1 = OneHotEncoder()
cat_transformer_2 = OrdinalEncoder

preproc = make_column_transformer(
    (num_transformer, ['age', 'bmi','avg_glucose_level']),
    (cat_transformer_1, ['gender', 'ever_married']),
    (cat_transformer_2, ['work_type','Residence_type','smoking_status']),
    remainder='passthrough'
)

In [36]:
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(prepoc, LogisticRegression())
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['age', 'bmi',
                                                   'avg_glucose_level']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['gender', 'ever_married']),
                                                 ('type',
                                                  <class 'sklearn.preprocessing._encoders.OrdinalEncoder'>,
                                                  ['work_type',
                                                   'Residence_type',
                                                   'smoking_status'])])),
                ('logisticregression', LogisticRegression())])

In [37]:
# Train Pipeline
pipeline.fit(X_train,y_train)

# Make predictions
pipeline.predict(X_test.iloc[0:1])

# Score model
pipeline.score(X_test,y_test)

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.